# 파일 소개 
- **파일 내용 : 부정지수, 중립지수, 긍정지수를 각각 계산하여 추가한 news_sentiment_final.xlsx 파일을 회사별로 분리한 뒤, 날짜별로 정리한 시계열 데이터를 만든다** 
- **1) 각 뉴스의 제목을 이용해 어느 회사에 대한 뉴스인지를 COMPANY 라는 새로운 변수에 저장하고, 이 변수를 기준으로 데이터프레임을 쪼갠다**
- **2) 쪼갠 회사별 데이터프레임은 주식 시장 개장시간을 고려하여 일자별로 POSITIVE-NEGATIVE 지수의 총합을 계산한다**

# 패키지, 데이터 로드

In [1]:
import pandas as pd 
import numpy as np
import re
import datetime


In [2]:
# 데이터 로드 
news = pd.read_excel("../../../data/News/news_sentiment_final.xlsx")
news.head() # 확인 

,NEWS_YM,DATE_TIME,WRITER,SUBCATEGORY,TITLE,IMPORTANCE,ITEM_NAME,TAG_LIST,PARSED CONTENT,LENGTH_SUM,SUMMARY,SUMMARY_2,SENTIMENT_NEGATIVE,SENTIMENT_NEUTRAL,SENTIMENT_POSITIVE
0,202303,20230303_134044,매일경제,기술,"하이브의 SM 인수, '적대적 M&A' 성격 두고 방시혁·SM 충돌",29.27,NaN,의사| 독점,방시혁 하이브 의장이 미국 CNN 방송에 출연해 SM엔터테인먼트 인수전을 적대적 M...,2326,해외 언론 인터뷰를 통해 방 의장의 입장이 알려진 건 지난 2월 10일 지분인수 발...,해외 언론 인터뷰를 통해 방 의장의 입장이 알려진 건 지분인수 발표 당일 이수만 전...,95.182945,4.809798,0.007258
1,202304,20230414_092237,eDaily,경제,"[특징주]엔터株, K-팝 약진 속 일제히 강세",27.41,JYP Ent.|하이브|와이지엔터테인먼트|에스엠,NaN,[이데일리 이용성 기자] 올해 2분기 국내 엔터주들의 합산 앨범판매량과 콘서트 관객...,2332,올해 2분기 국내 엔터주들의 합산 앨범판매량과 콘서트 관객수 모두 역대 최대 분기를...,올해 2분기 국내 엔터주들의 합산 앨범판매량과 콘서트 관객수 모두 역대 최대 분기를...,0.024469,0.035980,99.939550
2,202306,20230611_085817,eDaily,사회,"타다, 하이브 ‘위버스콘 페스티벌’서 K팝 팬 태워",24.31,NaN,NaN,- 10일부터 11일까지 양일간 페스티벌 참여- 올림픽공원과 롯데호텔 월드에 전용 ...,2308,10일부터 11일까지 양일간 페스티벌 참여- 올림픽공원과 롯데호텔 월드에 전용 탑승...,타다가 10일부터 11일까지 2023 ‘Weverse Con Festival’ (이...,0.012343,45.286148,54.701510
3,202301,20230102_084347,eDaily,사회,BTS 자리 넘본다…유망주는 스트레이 키즈-KB,67.91,NaN,BTS| 판매량| NCT| 넥스트| 엔하이픈| 블랙핑크| 에스파| 아이브,"- KB증권 “군 입대 BTS에 도전 이어질 것”- “스트레이 키즈, 군 이슈 없고...",1713,방탄소년단(BTS)을 이을 아이돌 그룹에 스트레이 키즈가 유력하다는 전망이 제기됐습...,방탄소년단(BTS)을 이을 아이돌 그룹에 스트레이 키즈가 유력하다는 전망이 제기됐습...,0.013367,7.872859,92.113780
4,202301,20230104_082655,eDaily,경제,BTS 군입대에도 K팝 주가 선방 전망-IBK,67.81,NaN,판매량| 방탄소년단| BTS| 개별| 공연| 글로벌,"- 이환욱 “K팝 앨범 판매 증가, 하이브 주가 상향”[이데일리 최훈길 기자] 방탄...",1887,방탄소년단(BTS) 군 입대에도 K팝 앨범과 주가가 선방할 것이란 전망이 제기됐습니...,방탄소년단(BTS) 군 입대에도 K팝 앨범과 주가가 선방할 것이란 전망이 제기됐습니...,32.145016,1.136920,66.718060


# 날짜 변수 전처리

### DATE_TIME 데이터 타입 변환

In [3]:
# DATE_TIME 변수를 datetime 형식으로 변경 

# DATE_TIME 열을 변환하는 함수 정의
def parse_date_time(date_time_str):
    year = int(date_time_str[:4])
    month = int(date_time_str[4:6])
    day = int(date_time_str[6:8])
    hour = int(date_time_str[9:11])
    minute = int(date_time_str[11:13])
    second = int(date_time_str[13:15])
    return pd.Timestamp(year=year, month=month, day=day, hour=hour, minute=minute, second=second)

# DATE_TIME 열을 변환
news['DATE_TIME'] = news['DATE_TIME'].apply(parse_date_time)

### 개장 시간대를 기준으로 한 일자변수 'DATE_NEW' 추가

In [4]:
date_list = []

for i in range(len(news)):
    if news["DATE_TIME"][i].time() >= datetime.time(15,30,0): #당일 장 마감시간(오후 3시 30분) 이후부터 자정까지
        date_new = news["DATE_TIME"][i] + datetime.timedelta(days=1) #익일 날짜로 설정
        date_list.append(date_new.date())
    else:
        date_list.append(news["DATE_TIME"][i].date())
    
news["DATE_NEW"] = date_list
news[["DATE_TIME", "DATE_NEW"]]

,DATE_TIME,DATE_NEW
0,2023-03-03 13:40:44,2023-03-03
1,2023-04-14 09:22:37,2023-04-14
2,2023-06-11 08:58:17,2023-06-11
3,2023-01-02 08:43:47,2023-01-02
4,2023-01-04 08:26:55,2023-01-04
...,...,...
2884,2023-06-14 09:24:26,2023-06-14
2885,2023-06-14 21:39:37,2023-06-15
2886,2023-06-01 09:47:38,2023-06-01
2887,2023-06-22 07:51:05,2023-06-22


### 요일 변수 추가 
향후 분석에 용이하도록 DATE_NEW 일자를 기준으로 일자별 요일을 지정해줌.
- 2023년 1월은 일요일부터 시작함

In [5]:
weekday_dict = {0:'월', 1:'화', 2:'수', 3:'목', 4:'금', 5:'토', 6:'일'}

In [6]:
day_list = []

for i in range(len(news)):
    day_list.append(weekday_dict[news["DATE_NEW"][i].weekday()])

news["DAY"] = day_list
news.groupby("DAY").size()

DAY
금    612
목    429
수    494
월    467
일    174
토    241
화    472
dtype: int64

In [7]:
#요일 배정이 잘 되었는지 확인
for day in weekday_dict.values():
    print(day, news[news["DAY"] == day]["DATE_NEW"].unique())

월 [datetime.date(2023, 1, 2) datetime.date(2023, 1, 9)
 datetime.date(2023, 1, 16) datetime.date(2023, 1, 23)
 datetime.date(2023, 1, 30) datetime.date(2023, 2, 13)
 datetime.date(2023, 2, 20) datetime.date(2023, 2, 27)
 datetime.date(2023, 2, 6) datetime.date(2023, 3, 6)
 datetime.date(2023, 3, 13) datetime.date(2023, 3, 27)
 datetime.date(2023, 3, 20) datetime.date(2023, 4, 3)
 datetime.date(2023, 4, 24) datetime.date(2023, 4, 17)
 datetime.date(2023, 4, 10) datetime.date(2023, 5, 15)
 datetime.date(2023, 5, 22) datetime.date(2023, 5, 8)
 datetime.date(2023, 5, 1) datetime.date(2023, 5, 29)
 datetime.date(2023, 6, 12) datetime.date(2023, 6, 5)
 datetime.date(2023, 6, 19)]
화 [datetime.date(2023, 1, 31) datetime.date(2023, 1, 3)
 datetime.date(2023, 1, 10) datetime.date(2023, 1, 17)
 datetime.date(2023, 1, 24) datetime.date(2023, 2, 14)
 datetime.date(2023, 2, 28) datetime.date(2023, 2, 21)
 datetime.date(2023, 2, 7) datetime.date(2023, 3, 7)
 datetime.date(2023, 3, 14) datetime.date(2

- **설명**
- '요일' 변수는 각 뉴스가 영향을 미치는 요일을 의미한다 
- ex1) 요일이 '금'으로 지정된 데이터 : 금요일 주가에 영향을 미치는 데이터 
- ex2) 요일이 '토'으로 지정된 데이터 : 금요일 주가에 영향을 미치는 데이터 


### 토요일, 일요일 데이터 전처리

- 토요일과 일요일 데이터는 ***월요일*** 주가에 영향을 미침
- 따라서 요일 변수가 토요일, 일요일로 설정된 데이터들은 그 다음주 월요일 데이터로 간주하여 날짜 및 요일 변수를 변경함

In [8]:
 for i in range(len(news)):
    if (news['DAY'].iloc[i] == '토'): # 토요일 뉴스기사라면, 
        news['DATE_NEW'].iloc[i]+= datetime.timedelta(days=2)  # DATE_NEW(날짜)변수에 이틀을 추가
        news['DAY'].iloc[i] = '월'       #DAY(요일) 변수를 월요일로
    if (news['DAY'].iloc[i] == '일'): # 일요일 뉴스기사라면, 
        news['DATE_NEW'].iloc[i]+= datetime.timedelta(days=1)  # DATE_NEW(날짜)변수에 하루를 추가
        news['DAY'].iloc[i] = '월'       #DAY(요일) 변수를 월요일로

/var/folders/4t/dyhc70gd7zjdwxdh7x7f77240000gn/T/ipykernel_14969/2229788862.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news['DATE_NEW'].iloc[i]+= datetime.timedelta(days=1)  # DATE_NEW(날짜)변수에 하루를 추가
/var/folders/4t/dyhc70gd7zjdwxdh7x7f77240000gn/T/ipykernel_14969/2229788862.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news['DAY'].iloc[i] = '월'       #DAY(요일) 변수를 월요일로
/var/folders/4t/dyhc70gd7zjdwxdh7x7f77240000gn/T/ipykernel_14969/2229788862.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pa

In [9]:
news.head(2)  # 확인

,NEWS_YM,DATE_TIME,WRITER,SUBCATEGORY,TITLE,IMPORTANCE,ITEM_NAME,TAG_LIST,PARSED CONTENT,LENGTH_SUM,SUMMARY,SUMMARY_2,SENTIMENT_NEGATIVE,SENTIMENT_NEUTRAL,SENTIMENT_POSITIVE,DATE_NEW,DAY
0,202303,2023-03-03 13:40:44,매일경제,기술,"하이브의 SM 인수, '적대적 M&A' 성격 두고 방시혁·SM 충돌",29.27,NaN,의사| 독점,방시혁 하이브 의장이 미국 CNN 방송에 출연해 SM엔터테인먼트 인수전을 적대적 M...,2326,해외 언론 인터뷰를 통해 방 의장의 입장이 알려진 건 지난 2월 10일 지분인수 발...,해외 언론 인터뷰를 통해 방 의장의 입장이 알려진 건 지분인수 발표 당일 이수만 전...,95.182945,4.809798,0.007258,2023-03-03,금
1,202304,2023-04-14 09:22:37,eDaily,경제,"[특징주]엔터株, K-팝 약진 속 일제히 강세",27.41,JYP Ent.|하이브|와이지엔터테인먼트|에스엠,NaN,[이데일리 이용성 기자] 올해 2분기 국내 엔터주들의 합산 앨범판매량과 콘서트 관객...,2332,올해 2분기 국내 엔터주들의 합산 앨범판매량과 콘서트 관객수 모두 역대 최대 분기를...,올해 2분기 국내 엔터주들의 합산 앨범판매량과 콘서트 관객수 모두 역대 최대 분기를...,0.024469,0.035980,99.939550,2023-04-14,금


### DATE_NEW 변수타입 변환

In [10]:
# 'DATE_NEW' 열을 datetime 형식으로 변환
news['DATE_NEW'] = pd.to_datetime(news['DATE_NEW'])

# 감정지수 변수 추가하기
* *****SENTIMENT (= POSITIVE - NEGATIVE) 변수*****

In [11]:
# POSITIVE - NEGATIVE 변수 구하기 
news['SENTIMENT'] = news['SENTIMENT_POSITIVE'] - news['SENTIMENT_NEGATIVE']

In [12]:
news.head(2) # 결과 확인 

,NEWS_YM,DATE_TIME,WRITER,SUBCATEGORY,TITLE,IMPORTANCE,ITEM_NAME,TAG_LIST,PARSED CONTENT,LENGTH_SUM,SUMMARY,SUMMARY_2,SENTIMENT_NEGATIVE,SENTIMENT_NEUTRAL,SENTIMENT_POSITIVE,DATE_NEW,DAY,SENTIMENT
0,202303,2023-03-03 13:40:44,매일경제,기술,"하이브의 SM 인수, '적대적 M&A' 성격 두고 방시혁·SM 충돌",29.27,NaN,의사| 독점,방시혁 하이브 의장이 미국 CNN 방송에 출연해 SM엔터테인먼트 인수전을 적대적 M...,2326,해외 언론 인터뷰를 통해 방 의장의 입장이 알려진 건 지난 2월 10일 지분인수 발...,해외 언론 인터뷰를 통해 방 의장의 입장이 알려진 건 지분인수 발표 당일 이수만 전...,95.182945,4.809798,0.007258,2023-03-03,금,-95.175687
1,202304,2023-04-14 09:22:37,eDaily,경제,"[특징주]엔터株, K-팝 약진 속 일제히 강세",27.41,JYP Ent.|하이브|와이지엔터테인먼트|에스엠,NaN,[이데일리 이용성 기자] 올해 2분기 국내 엔터주들의 합산 앨범판매량과 콘서트 관객...,2332,올해 2분기 국내 엔터주들의 합산 앨범판매량과 콘서트 관객수 모두 역대 최대 분기를...,올해 2분기 국내 엔터주들의 합산 앨범판매량과 콘서트 관객수 모두 역대 최대 분기를...,0.024469,0.035980,99.939550,2023-04-14,금,99.915081


# 회사별로 뉴스 데이터 분류

## 회사별 키워드 지정 

* *****HYBE*****

In [13]:
hybe_kw =  ['하이브','hybe','방시혁','뉴진스','new jeans','newjeans','민지','해린','혜인','다니엘','하니',
            '민희진','방탄소년단','bts','전정국','제이홉','rm','슈가','정국',
            '르세라핌','le sserafim','채원','카즈하','은채','사쿠라','허윤진',
            '세븐틴','seventeen','투모로우바이투게더', 'txt','프로미스나인','fromis 9','fromis9',
            '나나','nana','이현','lee hyun','미드낫','midnatt','엔하이픈','enhypen',
            '백호','강동호','황민현','예하나','성연','지코','zico','앤팀','&team','히라테 유리나',
            '보이넥스트도어','boynextdoor','boy next door','위버스','부석순']

* *****SM*****

In [14]:
sm_kw = ['sm','에스엠','이수만','boa','에스파','aespa','카리나','윈터','지젤','닝닝',
         '엑소','exo','샤이니','shinee','엔시티','nct',
         '동방신기','tvxq','슈퍼주니어','super junior','superjunior','소녀시대',
         '레드벨벳','red velvet','redvelvet','웨이션브이','슈퍼엠','superm','super m',
         '갓더비트','갓 더 비트','got the beat','라이즈','riize','디어유']

* *****YG*****

In [15]:
yg_kw = ['와이지','yg','양현석','블랙핑크','blackpink','black pink','제니','리사','지수','로제',
         '트레저','treasure','악뮤','악동뮤지션','akmu','위너','winnter','지디','GD','권지용','지드래곤',
         '젝스키스','sechskies','sechs kies','베이비몬스터','베이비 몬스터','babymonster','baby monster']

* *****JYP*****

In [16]:
jyp_kw = ['제이와이피','jyp','박진영','스트레이키즈','스트레이 키즈','straykids','stray kids',
          '트와이스','twice','엔믹스','nmixx', '데이식스','day6','day 6',
          '있지','잇지','itzy','채령','유나','류진','리아','예지',
          '투피엠','2pm','2 pm','보이스토리','boystory','boy story',
          '야오천','yaochen','니지유','niziu','엑스디너리 히어로즈','xdinary heroes','xdinaryheroes']

## COMPANY 변수 추가 
- 기준 : 제목에 어느 회사의 키워드가 가장 ‘먼저’ 등장하는지

In [17]:
news['COMPANY'] = ""

In [18]:
# 회사별 키워드 목록 및 인덱스 열 생성
company_keywords = {
    'HYBE': hybe_kw,
    'SM': sm_kw,
    'YG': yg_kw,
    'JYP': jyp_kw,
}
for company in company_keywords:
    news[f'{company}_KW_INDEX'] = -1

# 키워드의 인덱스를 찾는 함수
def find_keyword_index(title, keywords):
    index = len(title)  # 초기값을 title 문자열의 길이로 설정
    for keyword in keywords:
        keyword_index = title.lower().find(keyword.lower())
        if keyword_index != -1 and keyword_index < index:
            index = keyword_index
    return index

# 각 회사별 키워드의 인덱스 찾고 {회사명}_KW_INDEX 변수로 저장 
for company, keywords in company_keywords.items():
    news[f'{company}_KW_INDEX'] = news['TITLE'].apply(lambda x: find_keyword_index(x, keywords))

In [19]:
news[['TITLE','COMPANY','HYBE_KW_INDEX','SM_KW_INDEX','YG_KW_INDEX','JYP_KW_INDEX']].head() # 결과 확인

,TITLE,COMPANY,HYBE_KW_INDEX,SM_KW_INDEX,YG_KW_INDEX,JYP_KW_INDEX
0,"하이브의 SM 인수, '적대적 M&A' 성격 두고 방시혁·SM 충돌",,0,5,37,37
1,"[특징주]엔터株, K-팝 약진 속 일제히 강세",,25,25,25,25
2,"타다, 하이브 ‘위버스콘 페스티벌’서 K팝 팬 태워",,4,28,28,28
3,BTS 자리 넘본다…유망주는 스트레이 키즈-KB,,0,26,26,16
4,BTS 군입대에도 K팝 주가 선방 전망-IBK,,0,25,25,25


In [20]:
# {회사명}_KW_INDEX 변수 중 가장 작은 것을 COMPANY 에 할당. 모두 같을 경우 ALL 을 할당 
for i in range(len(news)):
    kw_indices = [news['HYBE_KW_INDEX'][i], news['SM_KW_INDEX'][i], news['YG_KW_INDEX'][i], news['JYP_KW_INDEX'][i]]
    min_index = min(kw_indices)
    
    if all(index == min_index for index in kw_indices):
        news.loc[i, 'COMPANY'] = 'ALL'
    elif min_index == news['HYBE_KW_INDEX'][i]:
        news.loc[i, 'COMPANY'] = 'HYBE'
    elif min_index == news['SM_KW_INDEX'][i]:
        news.loc[i, 'COMPANY'] = 'SM'
    elif min_index == news['YG_KW_INDEX'][i]:
        news.loc[i, 'COMPANY'] = 'YG'
    elif min_index == news['JYP_KW_INDEX'][i]:
        news.loc[i, 'COMPANY'] = 'JYP'

In [21]:
news.head(50) # 결과 확인 

,NEWS_YM,DATE_TIME,WRITER,SUBCATEGORY,TITLE,IMPORTANCE,ITEM_NAME,TAG_LIST,PARSED CONTENT,LENGTH_SUM,...,SENTIMENT_NEUTRAL,SENTIMENT_POSITIVE,DATE_NEW,DAY,SENTIMENT,COMPANY,HYBE_KW_INDEX,SM_KW_INDEX,YG_KW_INDEX,JYP_KW_INDEX
0,202303,2023-03-03 13:40:44,매일경제,기술,"하이브의 SM 인수, '적대적 M&A' 성격 두고 방시혁·SM 충돌",29.27,NaN,의사| 독점,방시혁 하이브 의장이 미국 CNN 방송에 출연해 SM엔터테인먼트 인수전을 적대적 M...,2326,...,4.809798,0.007258,2023-03-03,금,-95.175687,HYBE,0,5,37,37
1,202304,2023-04-14 09:22:37,eDaily,경제,"[특징주]엔터株, K-팝 약진 속 일제히 강세",27.41,JYP Ent.|하이브|와이지엔터테인먼트|에스엠,NaN,[이데일리 이용성 기자] 올해 2분기 국내 엔터주들의 합산 앨범판매량과 콘서트 관객...,2332,...,0.035980,99.939550,2023-04-14,금,99.915081,ALL,25,25,25,25
2,202306,2023-06-11 08:58:17,eDaily,사회,"타다, 하이브 ‘위버스콘 페스티벌’서 K팝 팬 태워",24.31,NaN,NaN,- 10일부터 11일까지 양일간 페스티벌 참여- 올림픽공원과 롯데호텔 월드에 전용 ...,2308,...,45.286148,54.701510,2023-06-12,월,54.689167,HYBE,4,28,28,28
3,202301,2023-01-02 08:43:47,eDaily,사회,BTS 자리 넘본다…유망주는 스트레이 키즈-KB,67.91,NaN,BTS| 판매량| NCT| 넥스트| 엔하이픈| 블랙핑크| 에스파| 아이브,"- KB증권 “군 입대 BTS에 도전 이어질 것”- “스트레이 키즈, 군 이슈 없고...",1713,...,7.872859,92.113780,2023-01-02,월,92.100413,HYBE,0,26,26,16
4,202301,2023-01-04 08:26:55,eDaily,경제,BTS 군입대에도 K팝 주가 선방 전망-IBK,67.81,NaN,판매량| 방탄소년단| BTS| 개별| 공연| 글로벌,"- 이환욱 “K팝 앨범 판매 증가, 하이브 주가 상향”[이데일리 최훈길 기자] 방탄...",1887,...,1.136920,66.718060,2023-01-04,수,34.573044,HYBE,0,25,25,25
5,202301,2023-01-15 10:37:41,eDaily,사회,제페토와 함께 아티스트 ‘태양’의 컴백을 즐겨보세요,57.65,NaN,제페토| 콘텐츠| 글로벌| 아이템| 플랫폼| 메타버스| 엔터테인먼트| 가상,- 글로벌 아티스트 태양 한정판 아이템 판매 및 틱톡 챌린지- 견고한 크리에이터 생...,1819,...,33.189068,66.803185,2023-01-16,월,66.795438,ALL,28,28,28,28
6,202301,2023-01-19 08:00:58,eDaily,사회,"와이지엔터, 블랙핑크 재계약 무난한 진행…우려 과도 -NH",66.27,와이지엔터테인먼트,블랙핑크| 음원,[이데일리 김소연 기자] NH투자증권은 19일 와이지엔터테인먼트(122870)에 대...,1938,...,7.701961,46.145590,2023-01-19,목,-0.006857,YG,32,32,0,32
7,202301,2023-01-03 19:06:35,eDaily,경제,"블랙핑크·뉴진스, 유튜브·음원 차트 점령…걸그룹 돌풍에 엔터株 훨훨",69.12,하이브|와이지엔터테인먼트,유튜브| 음원| 중국| 블랙핑크| 주력| 음반| 판매량| 공연,- [특징주]- 주력 걸그룹 인기몰이에 나홀로 승승장구- 한한령 완화 기대감도 투심...,1698,...,0.025601,66.639440,2023-01-04,수,33.304480,YG,5,37,0,37
8,202301,2023-01-20 07:54:07,eDaily,기술,"와이지엔터테인먼트, 베이비몬스터 데뷔 기대에 분위기 전환-메리츠",68.93,와이지엔터테인먼트,블랙핑크| 공연,- 양현석 사법 리스크 해소 및 베이비몬스터 흥행 기대- 지난해 4Q 실적은 컨센서...,1971,...,7.697240,92.287964,2023-01-20,금,92.273171,YG,35,35,0,35
9,202301,2023-01-04 13:46:39,연합뉴스,사회,"[가요소식] 트와이스, 영어 싱글 곡 20일 공개",29.00,NaN,트와이스| 글로벌| 스트리밍,"▲ 트와이스, 영어 싱글 곡 20일 공개 = 걸그룹 트와이스가 20일 영어 싱글 ...",1291,...,99.954990,0.029811,2023-01-04,수,0.014613,JYP,27,27,27,7


In [22]:
news = news.drop(['HYBE_KW_INDEX','SM_KW_INDEX','YG_KW_INDEX','JYP_KW_INDEX'],axis=1)

## COMPANY 변수를 기준으로 데이터 쪼개기

* *****HYBE*****

In [23]:
news_sentiment_hybe = news[(news['COMPANY'] == 'HYBE') | (news['COMPANY'] == 'ALL')]
news_sentiment_hybe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1476 entries, 0 to 2888
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   NEWS_YM             1476 non-null   int64         
 1   DATE_TIME           1476 non-null   datetime64[ns]
 2   WRITER              1476 non-null   object        
 3   SUBCATEGORY         1476 non-null   object        
 4   TITLE               1476 non-null   object        
 5   IMPORTANCE          1476 non-null   float64       
 6   ITEM_NAME           650 non-null    object        
 7   TAG_LIST            1206 non-null   object        
 8   PARSED CONTENT      1476 non-null   object        
 9   LENGTH_SUM          1476 non-null   int64         
 10  SUMMARY             1476 non-null   object        
 11  SUMMARY_2           1476 non-null   object        
 12  SENTIMENT_NEGATIVE  1476 non-null   float64       
 13  SENTIMENT_NEUTRAL   1476 non-null   float64     

* *****SM*****

In [24]:
news_sentiment_sm = news[(news['COMPANY'] == 'SM') | (news['COMPANY'] == 'ALL')]
news_sentiment_sm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1496 entries, 1 to 2887
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   NEWS_YM             1496 non-null   int64         
 1   DATE_TIME           1496 non-null   datetime64[ns]
 2   WRITER              1496 non-null   object        
 3   SUBCATEGORY         1496 non-null   object        
 4   TITLE               1496 non-null   object        
 5   IMPORTANCE          1496 non-null   float64       
 6   ITEM_NAME           817 non-null    object        
 7   TAG_LIST            1105 non-null   object        
 8   PARSED CONTENT      1496 non-null   object        
 9   LENGTH_SUM          1496 non-null   int64         
 10  SUMMARY             1496 non-null   object        
 11  SUMMARY_2           1496 non-null   object        
 12  SENTIMENT_NEGATIVE  1496 non-null   float64       
 13  SENTIMENT_NEUTRAL   1496 non-null   float64     

* *****YG*****

In [25]:
news_sentiment_yg = news[(news['COMPANY'] == 'YG') | (news['COMPANY'] == 'ALL')]
news_sentiment_yg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 669 entries, 1 to 2887
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   NEWS_YM             669 non-null    int64         
 1   DATE_TIME           669 non-null    datetime64[ns]
 2   WRITER              669 non-null    object        
 3   SUBCATEGORY         669 non-null    object        
 4   TITLE               669 non-null    object        
 5   IMPORTANCE          669 non-null    float64       
 6   ITEM_NAME           361 non-null    object        
 7   TAG_LIST            607 non-null    object        
 8   PARSED CONTENT      669 non-null    object        
 9   LENGTH_SUM          669 non-null    int64         
 10  SUMMARY             669 non-null    object        
 11  SUMMARY_2           669 non-null    object        
 12  SENTIMENT_NEGATIVE  669 non-null    float64       
 13  SENTIMENT_NEUTRAL   669 non-null    float64      

* *****JYP*****

In [26]:
news_sentiment_jyp = news[(news['COMPANY'] == 'JYP') | (news['COMPANY'] == 'ALL')]
news_sentiment_jyp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 628 entries, 1 to 2887
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   NEWS_YM             628 non-null    int64         
 1   DATE_TIME           628 non-null    datetime64[ns]
 2   WRITER              628 non-null    object        
 3   SUBCATEGORY         628 non-null    object        
 4   TITLE               628 non-null    object        
 5   IMPORTANCE          628 non-null    float64       
 6   ITEM_NAME           280 non-null    object        
 7   TAG_LIST            567 non-null    object        
 8   PARSED CONTENT      628 non-null    object        
 9   LENGTH_SUM          628 non-null    int64         
 10  SUMMARY             628 non-null    object        
 11  SUMMARY_2           628 non-null    object        
 12  SENTIMENT_NEGATIVE  628 non-null    float64       
 13  SENTIMENT_NEUTRAL   628 non-null    float64      

## 엑셀 파일로 저장 

In [27]:
# 엑셀 파일로 저장 
excel_filename = '../../../data/News/news_classified_hybe.xlsx'
news_sentiment_hybe.to_excel(excel_filename, index=False)

In [28]:
# 엑셀 파일로 저장 
excel_filename = '../../../data/News/news_classified_sm.xlsx'
news_sentiment_sm.to_excel(excel_filename, index=False)

In [29]:
# 엑셀 파일로 저장 
excel_filename = '../../../data/News/news_classified_yg.xlsx'
news_sentiment_yg.to_excel(excel_filename, index=False)

In [30]:
# 엑셀 파일로 저장 
excel_filename = '../../../data/News/news_classified_jyp.xlsx'
news_sentiment_jyp.to_excel(excel_filename, index=False)

# 날짜별 감정지수 변수 데이터프레임 생성

## HYBE

### 날짜별로 감정지수 변수 값 총합 구하기

In [31]:
news_sentiment_hybe.head(1) # 작업 전 확인

,NEWS_YM,DATE_TIME,WRITER,SUBCATEGORY,TITLE,IMPORTANCE,ITEM_NAME,TAG_LIST,PARSED CONTENT,LENGTH_SUM,SUMMARY,SUMMARY_2,SENTIMENT_NEGATIVE,SENTIMENT_NEUTRAL,SENTIMENT_POSITIVE,DATE_NEW,DAY,SENTIMENT,COMPANY
0,202303,2023-03-03 13:40:44,매일경제,기술,"하이브의 SM 인수, '적대적 M&A' 성격 두고 방시혁·SM 충돌",29.27,NaN,의사| 독점,방시혁 하이브 의장이 미국 CNN 방송에 출연해 SM엔터테인먼트 인수전을 적대적 M...,2326,해외 언론 인터뷰를 통해 방 의장의 입장이 알려진 건 지난 2월 10일 지분인수 발...,해외 언론 인터뷰를 통해 방 의장의 입장이 알려진 건 지분인수 발표 당일 이수만 전...,95.182945,4.809798,0.007258,2023-03-03,금,-95.175687,HYBE


In [32]:
# 'DATE_NEW'를 기준으로 그룹화하여 'SENTIMENT'의 합 계산
hybe_df = news_sentiment_hybe.groupby('DATE_NEW').agg({'SENTIMENT': 'sum', 'DAY': 'first'}).reset_index()

# 열 이름 변경
hybe_df = hybe_df.rename(columns={'DATE_NEW': 'DATE', 'SENTIMENT': 'SENTIMENT_SUM', 'DAY': 'DAY'})

In [33]:
# 결과 출력
hybe_df

,DATE,SENTIMENT_SUM,DAY
0,2023-01-02,233.751368,월
1,2023-01-03,181.407140,화
2,2023-01-04,381.612804,수
3,2023-01-05,558.004627,목
4,2023-01-06,413.298417,금
...,...,...,...
121,2023-06-20,229.447152,화
122,2023-06-21,74.912883,수
123,2023-06-22,664.385349,목
124,2023-06-23,164.706250,금


### 월요일의 감정지수 변수는 3으로 나눠주기 

In [34]:
for i in range(len(hybe_df)):
    if hybe_df['DAY'].iloc[i] == '월':
        hybe_df['SENTIMENT_SUM'].iloc[i] = (hybe_df['SENTIMENT_SUM'].iloc[i])/3

/var/folders/4t/dyhc70gd7zjdwxdh7x7f77240000gn/T/ipykernel_14969/2116828930.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hybe_df['SENTIMENT_SUM'].iloc[i] = (hybe_df['SENTIMENT_SUM'].iloc[i])/3


In [35]:
# 결과 확인 
hybe_df

,DATE,SENTIMENT_SUM,DAY
0,2023-01-02,77.917123,월
1,2023-01-03,181.407140,화
2,2023-01-04,381.612804,수
3,2023-01-05,558.004627,목
4,2023-01-06,413.298417,금
...,...,...,...
121,2023-06-20,229.447152,화
122,2023-06-21,74.912883,수
123,2023-06-22,664.385349,목
124,2023-06-23,164.706250,금


## SM
### 날짜별로 감정지수 변수 값 총합 구하기

In [36]:
news_sentiment_sm.head(1) # 작업 전 확인

,NEWS_YM,DATE_TIME,WRITER,SUBCATEGORY,TITLE,IMPORTANCE,ITEM_NAME,TAG_LIST,PARSED CONTENT,LENGTH_SUM,SUMMARY,SUMMARY_2,SENTIMENT_NEGATIVE,SENTIMENT_NEUTRAL,SENTIMENT_POSITIVE,DATE_NEW,DAY,SENTIMENT,COMPANY
1,202304,2023-04-14 09:22:37,eDaily,경제,"[특징주]엔터株, K-팝 약진 속 일제히 강세",27.41,JYP Ent.|하이브|와이지엔터테인먼트|에스엠,NaN,[이데일리 이용성 기자] 올해 2분기 국내 엔터주들의 합산 앨범판매량과 콘서트 관객...,2332,올해 2분기 국내 엔터주들의 합산 앨범판매량과 콘서트 관객수 모두 역대 최대 분기를...,올해 2분기 국내 엔터주들의 합산 앨범판매량과 콘서트 관객수 모두 역대 최대 분기를...,0.024469,0.03598,99.93955,2023-04-14,금,99.915081,ALL


In [37]:
# 'DATE_NEW'를 기준으로 그룹화하여 'SENTIMENT'의 합 계산
sm_df = news_sentiment_sm.groupby('DATE_NEW').agg({'SENTIMENT': 'sum', 'DAY': 'first'}).reset_index()


In [38]:
# 열 이름 변경
sm_df = sm_df.rename(columns={'DATE_NEW': 'DATE', 'SENTIMENT': 'SENTIMENT_SUM', 'DAY': 'DAY'})

# 결과 출력
sm_df

,DATE,SENTIMENT_SUM,DAY
0,2023-01-02,242.321497,월
1,2023-01-03,181.916077,화
2,2023-01-04,21.082685,수
3,2023-01-05,277.915244,목
4,2023-01-06,318.234971,금
...,...,...,...
120,2023-06-20,169.185466,화
121,2023-06-21,74.912883,수
122,2023-06-22,159.179436,목
123,2023-06-23,12.319740,금


### 월요일의 감정지수 변수는 3으로 나눠주기

In [39]:
for i in range(len(sm_df)):
    if sm_df['DAY'].iloc[i] == '월':
        sm_df['SENTIMENT_SUM'].iloc[i] = (sm_df['SENTIMENT_SUM'].iloc[i])/3

# 결과 확인 
sm_df

/var/folders/4t/dyhc70gd7zjdwxdh7x7f77240000gn/T/ipykernel_14969/1946809480.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sm_df['SENTIMENT_SUM'].iloc[i] = (sm_df['SENTIMENT_SUM'].iloc[i])/3


,DATE,SENTIMENT_SUM,DAY
0,2023-01-02,80.773832,월
1,2023-01-03,181.916077,화
2,2023-01-04,21.082685,수
3,2023-01-05,277.915244,목
4,2023-01-06,318.234971,금
...,...,...,...
120,2023-06-20,169.185466,화
121,2023-06-21,74.912883,수
122,2023-06-22,159.179436,목
123,2023-06-23,12.319740,금


## YG
### 날짜별로 감정지수 변수 값 총합 구하기

In [40]:
news_sentiment_yg.head(1) # 작업 전 확인

,NEWS_YM,DATE_TIME,WRITER,SUBCATEGORY,TITLE,IMPORTANCE,ITEM_NAME,TAG_LIST,PARSED CONTENT,LENGTH_SUM,SUMMARY,SUMMARY_2,SENTIMENT_NEGATIVE,SENTIMENT_NEUTRAL,SENTIMENT_POSITIVE,DATE_NEW,DAY,SENTIMENT,COMPANY
1,202304,2023-04-14 09:22:37,eDaily,경제,"[특징주]엔터株, K-팝 약진 속 일제히 강세",27.41,JYP Ent.|하이브|와이지엔터테인먼트|에스엠,NaN,[이데일리 이용성 기자] 올해 2분기 국내 엔터주들의 합산 앨범판매량과 콘서트 관객...,2332,올해 2분기 국내 엔터주들의 합산 앨범판매량과 콘서트 관객수 모두 역대 최대 분기를...,올해 2분기 국내 엔터주들의 합산 앨범판매량과 콘서트 관객수 모두 역대 최대 분기를...,0.024469,0.03598,99.93955,2023-04-14,금,99.915081,ALL


In [41]:
# 'DATE_NEW'를 기준으로 그룹화하여 'SENTIMENT'의 합 계산
yg_df = news_sentiment_yg.groupby('DATE_NEW').agg({'SENTIMENT': 'sum', 'DAY': 'first'}).reset_index()

# 열 이름 변경
yg_df = yg_df.rename(columns={'DATE_NEW': 'DATE', 'SENTIMENT': 'SENTIMENT_SUM', 'DAY': 'DAY'})

# 결과 출력
yg_df

,DATE,SENTIMENT_SUM,DAY
0,2023-01-02,378.083107,월
1,2023-01-03,119.020563,화
2,2023-01-04,209.513567,수
3,2023-01-05,277.915244,목
4,2023-01-06,269.611104,금
...,...,...,...
121,2023-06-20,169.185466,화
122,2023-06-21,74.912883,수
123,2023-06-22,159.179436,목
124,2023-06-23,72.412366,금


### 월요일의 감정지수 변수는 3으로 나눠주기

In [42]:
for i in range(len(sm_df)):
    if yg_df['DAY'].iloc[i] == '월':
        yg_df['SENTIMENT_SUM'].iloc[i] = (yg_df['SENTIMENT_SUM'].iloc[i])/3

# 결과 확인 
yg_df

/var/folders/4t/dyhc70gd7zjdwxdh7x7f77240000gn/T/ipykernel_14969/2886694626.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yg_df['SENTIMENT_SUM'].iloc[i] = (yg_df['SENTIMENT_SUM'].iloc[i])/3


,DATE,SENTIMENT_SUM,DAY
0,2023-01-02,126.027702,월
1,2023-01-03,119.020563,화
2,2023-01-04,209.513567,수
3,2023-01-05,277.915244,목
4,2023-01-06,269.611104,금
...,...,...,...
121,2023-06-20,169.185466,화
122,2023-06-21,74.912883,수
123,2023-06-22,159.179436,목
124,2023-06-23,72.412366,금


## JYP
### 날짜별로 감정지수 변수 값 총합 구하기

In [43]:
news_sentiment_jyp.head(1) # 작업 전 확인


,NEWS_YM,DATE_TIME,WRITER,SUBCATEGORY,TITLE,IMPORTANCE,ITEM_NAME,TAG_LIST,PARSED CONTENT,LENGTH_SUM,SUMMARY,SUMMARY_2,SENTIMENT_NEGATIVE,SENTIMENT_NEUTRAL,SENTIMENT_POSITIVE,DATE_NEW,DAY,SENTIMENT,COMPANY
1,202304,2023-04-14 09:22:37,eDaily,경제,"[특징주]엔터株, K-팝 약진 속 일제히 강세",27.41,JYP Ent.|하이브|와이지엔터테인먼트|에스엠,NaN,[이데일리 이용성 기자] 올해 2분기 국내 엔터주들의 합산 앨범판매량과 콘서트 관객...,2332,올해 2분기 국내 엔터주들의 합산 앨범판매량과 콘서트 관객수 모두 역대 최대 분기를...,올해 2분기 국내 엔터주들의 합산 앨범판매량과 콘서트 관객수 모두 역대 최대 분기를...,0.024469,0.03598,99.93955,2023-04-14,금,99.915081,ALL


In [44]:
# 'DATE_NEW'를 기준으로 그룹화하여 'SENTIMENT'의 합 계산
jyp_df = news_sentiment_jyp.groupby('DATE_NEW').agg({'SENTIMENT': 'sum', 'DAY': 'first'}).reset_index()

# 열 이름 변경
jyp_df = jyp_df.rename(columns={'DATE_NEW': 'DATE', 'SENTIMENT': 'SENTIMENT_SUM', 'DAY': 'DAY'})

# 결과 출력
jyp_df

,DATE,SENTIMENT_SUM,DAY
0,2023-01-02,75.025892,월
1,2023-01-03,175.724840,화
2,2023-01-04,21.097298,수
3,2023-01-05,277.915244,목
4,2023-01-06,393.110160,금
...,...,...,...
119,2023-06-20,169.198873,화
120,2023-06-21,74.912883,수
121,2023-06-22,159.179436,목
122,2023-06-23,147.327727,금


### 월요일의 감정지수 변수는 3으로 나눠주기

In [45]:
for i in range(len(jyp_df)):
    if jyp_df['DAY'].iloc[i] == '월':
        jyp_df['SENTIMENT_SUM'].iloc[i] = (jyp_df['SENTIMENT_SUM'].iloc[i])/3

# 결과 확인 
jyp_df

/var/folders/4t/dyhc70gd7zjdwxdh7x7f77240000gn/T/ipykernel_14969/2051178185.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jyp_df['SENTIMENT_SUM'].iloc[i] = (jyp_df['SENTIMENT_SUM'].iloc[i])/3


,DATE,SENTIMENT_SUM,DAY
0,2023-01-02,25.008631,월
1,2023-01-03,175.724840,화
2,2023-01-04,21.097298,수
3,2023-01-05,277.915244,목
4,2023-01-06,393.110160,금
...,...,...,...
119,2023-06-20,169.198873,화
120,2023-06-21,74.912883,수
121,2023-06-22,159.179436,목
122,2023-06-23,147.327727,금


# 엑셀 파일로 저장

In [46]:
# 엑셀 파일로 저장 
excel_filename = '../../../data/News/sentiment_hybe.xlsx'
hybe_df.to_excel(excel_filename, index=False)

In [47]:
# 엑셀 파일로 저장 
excel_filename = '../../../data/News/sentiment_sm.xlsx'
sm_df.to_excel(excel_filename, index=False)

In [48]:
# 엑셀 파일로 저장 
excel_filename = '../../../data/News/sentiment_yg.xlsx'
yg_df.to_excel(excel_filename, index=False)

In [49]:
# 엑셀 파일로 저장 
excel_filename = '../../../data/News/sentiment_jyp.xlsx'
jyp_df.to_excel(excel_filename, index=False)